In [1]:
import random
import numpy as np

In [2]:
random.seed(42)
np.random.seed(42)

In [3]:
from cirkit.utils.scope import Scope
from cirkit.symbolic.parameters import LogSoftmaxParameter, ExpParameter, Parameter
from cirkit.symbolic.layers import CategoricalLayer, MixingLayer, HadamardLayer
from cirkit.symbolic.initializers import NormalInitializer
from cirkit.symbolic.circuit import Circuit

In [4]:
def categorical_layer_factory(
    scope: Scope,
    num_units: int,
    num_channels: int,
    num_categories: int
) -> CategoricalLayer:
    return CategoricalLayer(
        scope, num_units, num_channels, num_categories=num_categories,
        parameterization=lambda p: Parameter.from_unary(p, LogSoftmaxParameter(p.shape)),
        initializer=NormalInitializer(0.0, 1e-2)
    )

def mixing_layer_factory(
    scope: Scope, num_units: int, arity: int
) -> MixingLayer:
    return MixingLayer(
        scope, num_units, arity,
        parameterization=lambda p: Parameter.from_unary(p, ExpParameter(p.shape)),
        initializer=NormalInitializer(0.0, 1e-1)
    )

def hadamard_layer_factory(scope: Scope, num_input_units: int, arity: int) -> HadamardLayer:
    return HadamardLayer(scope, num_input_units, arity)

In [5]:
n_variables = 13
n_channels = 3
n_categories = 17
n_components = 11

layers = [
    categorical_layer_factory(range(n_variables // 2), 1, n_channels, n_categories)
    for _ in range(n_components)
]
layers.extend(
    [
        categorical_layer_factory(range(n_variables // 2, n_variables), 1, n_channels, n_categories)
        for _ in range(n_components)
    ]
)
layers.append(mixing_layer_factory(range(n_variables // 2), 1, n_components))
layers.append(mixing_layer_factory(range(n_variables // 2, n_variables), 1, n_components))
layers.append(hadamard_layer_factory(range(n_variables), 2, n_components))

in_layers = dict()
in_layers[layers[-3]] = layers[:n_components]
in_layers[layers[-2]] = layers[n_components : 2 * n_components]
in_layers[layers[-1]] = layers[-3:-1]

out_layers = dict()
for i in range(n_components):
    out_layers[layers[i]] = [layers[-3]]
    out_layers[layers[i + n_components]] = [layers[-2]]
out_layers[layers[-3]] = [layers[-1]]
out_layers[layers[-2]] = [layers[-1]]

circuit = Circuit(
    scope=range(n_variables),
    num_channels=n_channels,
    layers=layers,
    in_layers=in_layers,
    out_layers=out_layers,
    topologically_ordered=True
)

In [6]:
import torch
device = torch.device('cuda')  # The device to use
torch.manual_seed(42)
if 'cuda' in device.type:
    torch.cuda.manual_seed(42)
    
from cirkit.pipeline import PipelineContext

Start testing with unfolded circuits, implementation will be more intuitive and has to be there anyway.

In [7]:
ctx = PipelineContext(
    backend='torch',
    fold=False,
    semiring='lse-sum'
)
circuit = ctx.compile(circuit)

In [8]:
print(len(list(circuit.layers)))

25


In [9]:
print(circuit)

TorchCircuit(
  (_nodes): ModuleList(
    (0-21): 22 x TorchCategoricalLayer(
      (logits): TorchParameter(
        (_nodes): ModuleList(
          (0): TorchTensorParameter()
          (1): TorchLogSoftmaxParameter()
        )
      )
    )
    (22-23): 2 x TorchMixingLayer(
      (weight): TorchParameter(
        (_nodes): ModuleList(
          (0): TorchTensorParameter()
          (1): TorchExpParameter()
        )
      )
    )
    (24): TorchHadamardLayer()
  )
)


In [10]:
samples = circuit.sample_forward(100)

print(samples.shape)

log_samples = circuit(samples)

print(log_samples.shape)

torch.Size([100, 3, 13])
torch.Size([1, 100])
